# Train token classification

- Author: Didier Guillevic
- Date: 2024-08-10
- Code: mostly copy/pasted from
[HuggingFace's course on token classification](https://huggingface.co/learn/nlp-course/en/chapter7/2)

In [ ]:
from datasets import load_dataset

## Dataset

### Load dataset

In [ ]:
data_files = {
	"train": "openaddresses_ca_preprocessed_token_classif_train.parquet",
	"validation": "openaddresses_ca_preprocessed_token_classif_validation.parquet",
	"test": "openaddresses_ca_preprocessed_token_classif_test.parquet"
}
dataset = load_dataset("parquet", data_files=data_files)
dataset

Load the label names...

In [ ]:
import json

# Load the list from the file
with open("labels.json", "r") as f:
    label_names = json.load(f)

print(label_names)

### Tokenize dataset

In [ ]:
dataset['train'][1]

In [ ]:
from transformers import AutoTokenizer

model_checkpoint = "google-bert/bert-base-multilingual-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
tokenizer.is_fast

In [ ]:
inputs = tokenizer(dataset['train'][1]['words'], is_split_into_words=True )
inputs.tokens()

In [ ]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [ ]:
labels = dataset['train'][1]['labels']
word_ids = inputs.word_ids()
print(labels)
print(align_labels_with_tokens(labels, word_ids))

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["words"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["labels"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [ ]:
dataset['train'].column_names

In [ ]:
dataset_tokenized = dataset.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=dataset['train'].column_names,
)

In [ ]:
dataset_tokenized

## Model finetuning for token classification

### Data collation

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [ ]:
batch = data_collator([dataset_tokenized['train'][i] for i in range(2)])
batch["labels"]

In [ ]:
for i in range(2):
    print(dataset_tokenized['train'][i]["labels"])

### Metrics

In [ ]:
import evaluate

metric = evaluate.load("seqeval")

Note that the seqeval metrics expects the lists of labels as strings.

In [ ]:
labels = dataset['train'][0]["labels"]
labels = [label_names[i] for i in labels]
labels

In [ ]:
# Let's test with some fake predictions
predictions = labels.copy()
predictions[2] = "O"
metric.compute(predictions=[predictions], references=[labels])

In [ ]:
import numpy as np

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "CITY_f1": all_metrics['CITY']['f1'],
        "POSTCODE_f1": all_metrics['POSTCODE']['f1'],
        "REGION_f1": all_metrics['REGION']['f1'],
        "STREET_NAME_f1": all_metrics['STREET_NAME']['f1'],
        "STREET_NB_f1": all_metrics['STREET_NB']['f1'],
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

### Define model

In [ ]:
id2label = {i: label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

In [ ]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)

In [ ]:
model.config.num_labels

### Fine-tuning the model

In [ ]:
from huggingface_hub import notebook_login

#notebook_login()

In [ ]:
from transformers import TrainingArguments
import accelerate

In [ ]:
args = TrainingArguments(
    "bert-base-multilingual-uncased-finetuned-postal-can",
    evaluation_strategy="steps",
    eval_steps=25_000,
    save_strategy="steps",
    save_steps=25_000,
    save_total_limit=5,
    learning_rate=2e-5,
    num_train_epochs=1,
    weight_decay=0.01,
    push_to_hub=False,
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=dataset_tokenized["train"],
    eval_dataset=dataset_tokenized["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)
trainer.train()

In [ ]:
trainer.push_to_hub(commit_message="Training complete")

## Using the model

In [ ]:
from transformers import pipeline

# Replace this with your own checkpoint
model_checkpoint = "Didier/bert-base-multilingual-uncased-finetuned-postal-can"
token_classifier = pipeline(
    "token-classification", model=model_checkpoint, aggregation_strategy="simple"
)

In [ ]:
token_classifier("1 Sussex Dr, Ottawa, ON K1A 0A1")